# Forward Mode Automatic Differentiation
## Operator Overloading Approach

In this notebook, we will implement **Forward Mode Automatic Differentiation** using the operator overloading method in Python. The core idea is to define a custom `Variable` class that keeps track of both the value and its derivative (the primal and tangent) and overloads arithmetic operations to propagate derivatives according to calculus rules.


## 📐 The `Variable` Class
The `Variable` class encapsulates two values:
- `primal`: the actual value of the variable (used during function evaluation)
- `tangent`: the derivative of the function with respect to that variable

We also overload arithmetic operations so that when two `Variable` instances interact, both the value and the derivative are computed automatically.


In [58]:
import math

class Variable:
    def __init__(self, primal, tangent=0.0):
        self.primal = primal
        self.tangent = tangent

    def __add__(self, other):
        return Variable(self.primal + other.primal, self.tangent + other.tangent)

    def __sub__(self, other):
        return Variable(self.primal - other.primal, self.tangent - other.tangent)

    def __mul__(self, other):
        return Variable(
            self.primal * other.primal,
            self.tangent * other.primal + other.tangent * self.primal
        )

    def __truediv__(self, other):
        return Variable(
            self.primal / other.primal,
            (self.tangent * other.primal - self.primal * other.tangent) / (other.primal**2)
        )
    
    def __radd__(self, other):
        return self + other if isinstance(other, Variable) else Variable(self.primal + other, self.tangent)

    def __rmul__(self, other):
        return self * other if isinstance(other, Variable) else Variable(self.primal * other, self.tangent * other)


    def __repr__(self):
        return f"primal: {self.primal:.3f}, tangent: {self.tangent:.3f}"
        

def sin(x):
    return Variable(math.sin(x.primal), math.cos(x.primal) * x.tangent)

def exp(x):
    epx = math.exp(x.primal)
    return Variable(epx, epx * x.tangent)

def square(x):
    return Variable(x.primal**2, 2 * x.primal * x.tangent)

    


## Example 1: Automatic differentiation in forward mode
Let's calculate the derivative of the function

$$f(x_1, x_2) = \left[\sin\left(\frac{x_1}{x_2}\right) + \frac{x_1}{x_2} - e^{x_2}\right] \cdot \left[\frac{x_1}{x_2} - e^{x_2}\right]$$

using forward mode, at the point $ (x_1, x_2) = (1.5, 0.5) $ first with respect to $ x_1 $, and then with respect to $ x_2 $.

In [37]:
def f(x1, x2):
    return (sin(x1 / x2) + x1 / x2 - exp(x2)) * (x1 / x2 - exp(x2))

### Calculation of $ \frac{\partial f}{\partial x_1} $ at (1.5, 0.5)

In [38]:
x1 = Variable(1.5, 1.0)  # ∂x1/∂x1 = 1
x2 = Variable(0.5, 0.0)  # ∂x2/∂x1 = 0

y = f(x1, x2)
print(f"f(x1, x2) = {y.primal:.3f}")
print(f"∂f/∂x1 = {y.tangent:.3f}")

f(x1, x2) = 2.017
∂f/∂x1 = 3.012


### Calculation of $ \frac{\partial f}{\partial x_2} $ at (1.5, 0.5)

In [39]:
x1 = Variable(1.5, 0.0)  # ∂x1/∂x2 = 0
x2 = Variable(0.5, 1.0)  # ∂x2/∂x2 = 1

y = f(x1, x2)
print(f"f(x1, x2) = {y.primal:.3f}")
print(f"∂f/∂x2 = {y.tangent:.3f}")


f(x1, x2) = 2.017
∂f/∂x2 = -13.724


## Example 2: Automatic differentiation in forward mode

Let's calculate the derivative of the function



\begin{equation}

    f(x_1, x_2, x_3, x_4) = (x_2 \sin(x_1) + x_2^2, 2 \, x_3x_4 + x_1) = (r_0, r_1)

\end{equation}



using forward mode, at the point $(x_1, x_2, x_3, x_4) = (1.5, 0.5, 2.0, 3.0)$. This example will focus on getting the Jacobian matrix.

In [110]:
def g(x1, x2, x3, x4):
    r0 = x2 * sin(x1) + square(x2)
    r1 = 2 * x3 * x4 + x1
    return [r0, r1]


In [138]:
def compute_jacobian(f, x_vals):
    n_inputs = len(x_vals)
    y_sample = f(*[Variable(val, 0.0) for val in x_vals])
    n_outputs = len(y_sample)
    
    jacobian = []

    for i in range(n_inputs):
        # Set all tangents to 0.0 except for the i-th variable
        x_vars = [Variable(val, 1.0 if j == i else 0.0) for j, val in enumerate(x_vals)]
        y = f(*x_vars)
        jacobian.append([yi.tangent for yi in y])
    
    # Print primal output and the Jacobian in aligned format with square brackets
    print(f"g({', '.join(map(str, x_vals))}) =", [yi.primal for yi in y_sample])
    print("\nJ_g:")
    for row in list(map(list, zip(*jacobian))):
        print(f"[{', '.join(f'{val: .3f}' for val in row)}]")

    return jacobian


In [139]:
x_vals = [1.5, 0.5, 2.0, 3.0]
J = compute_jacobian(g, x_vals)

g(1.5, 0.5, 2.0, 3.0) = [0.7487474933020273, 13.5]

J_g:
[ 0.035,  1.997,  0.000,  0.000]
[ 1.000,  0.000,  6.000,  4.000]


# Reverse Mode